# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("output_data/cities.csv")
cities_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity=cities_df["Humidity"].astype(float)
humidity.head()


0    72.0
1    74.0
2    73.0
3    39.0
4    76.0
Name: Humidity, dtype: float64

In [4]:
max_humidity= humidity.max()
max_humidity

290.0

In [5]:
locations= cities_df[["Lat","Lng"]]
locations.head()

,Lat,Lng
0,68.05,39.51
1,-20.41,57.70
2,77.48,-69.36
3,40.71,112.04
4,-33.93,18.42


In [6]:
fig=gmaps.figure()
heat_layer=gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=max_humidity,point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
fig = gmaps.figure(map_type="TERRAIN")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=10,point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
city_weather_df = cities_df.loc[(cities_df["Max Temp"] > 70)& (cities_df["Max Temp"] < 80)&(cities_df["Cloudiness"]==0), :]
city_weather_df = city_weather_df.dropna(how='any')
city_weather_df.reset_index(inplace=True)
city_weather_df

,index,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,56,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41
1,70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
2,88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
3,114,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86
4,138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
5,139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
6,176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
7,297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
8,391,391,namibe,0,AO,1558378870,74,-15.19,12.15,71.70,17.40
9,444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


In [10]:
del city_weather_df['index']
city_weather_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41
1,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
2,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
3,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86
4,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
5,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
6,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
7,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
8,391,namibe,0,AO,1558378870,74,-15.19,12.15,71.70,17.40
9,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df = city_weather_df.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "html_attributions": [],
    "next_page_token": "ATtYBwKpVRatvICKv09hhP3FkZAHmMhSUxTx8Xu64zuv2Sp51RTAiWTzD0_72UiOG5HkHdAwTPX0-0Yey8nkzrW-qBIP2Lj7lsS3Qz2NebL-4fm_QJ7SqPJ6lmbxOJp4AzfjFKj4wi-u13VwMDfC-gP9ShqWU1y7bTOjN7KtzzpIXRW-bslLzwrrcibpxkNkcsyGcQYD3TOXUOV1VO8Sc3yjbj6bV5xsuhk7gxj4lmVUpgX704FKTa6H74bUIJ9I6afhGtHtXD6AM25oGyLl9wybtuspNctnryehgdQxWbKalMIqAiJHgpDEeW9AlQbhj1L_uqo-2ctRr6KezPfWY1bmc7sR9eavrkmjU8EuQbRkBayq-x9uXvWxTBqoETwJERSLUJ5ha--944br1r2bR4IRRAuzzC63lTxTQa0E4SYYxWpxjmPXp2Aa9G7UVpnTg7FS",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -13.8362619,
                    "lng": -76.2541983
                },
                "viewport": {
                    "northeast": {
                        "lat": -13.83495102010728,
                        "lng": -76.25282897010729
                    },
                    "southwest": {
                        "lat": -13.8376

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJ_1gLMwGwvQo0kNjeT2lYhScdY5we_JLA3M7PRRKZnWWFyAePhes-AUmpQOP9qwbyBSl_7BURgaDreicqcp27Jph7Vgof8QeRRYVOL_Ls8j7yYanthCi47t1FOtQZz1WwyOC9BgRTHmmmIBI6HczxVwjQtjwoICRcs-PPE8YSvN3kjbNrhV8KIlG1wGHdeEz_2TIVXE2DyN_Tqc0g_xtUWGhqTKEv_A278Ey1ILSLb8ynnwApyv-0nxt6TuTdGAlZCgCWJw0bPCvERX4yqPbadNsfHUPGr6kuXCFVL8Jo6jZskRukqONS6p969ljrlE42q42SvJ6INmvfFmqvJryl-Dd4x9ffcJPEDRP_L7EB9q626lPjXmPtBAmFGEkVwSEQdiTfwLk-WuxlqM8c_Y1plABBUovjRV3hgV4-qotDc6icxRGThTnlpX-_IgLCU",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.576865,
                    "lng": -46.0574868
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.57485467010727,
                        "lng": -46.0523694
                    },
                    "southwest": {
                        "lat": -20.577554329892

{
    "html_attributions": [],
    "next_page_token": "ATtYBwIbY-ZoT63abZb-_S9N1J_V5-k1o5gdKiexQPKdADPs4qMTGNXofh7XgldBzeoqdWdwF4Baxlv7vsPQeFKJO0HaDJLJ7Y1Mw7XhgW6U8USaL-X3cIC8B9FK1X3yeRfANxcbbCsttPs-_9uGHKOvE_ZgtsUG3u6hwOyEu2xj5_FM64sZjMSJZp_pVwcPELCnwxP3t3IMEl622TobULluKgrVXfZhXcr3TP6GJ4EUudGecJbgvS6RL8Q6SXizRPUL8pkfrBIgS7CgXcjLH1ZwqlaGKDw1k9PBRH4yzOAWFgFd880ikbj5SUC4TxWR9LQjSX509yqgX5m7tc50OJCkSO_Dpnz4neKHpM59bLU_2k8HRQXicJoNnAOMeQ0Z2yYBiNMUup_TXhDf2_eqJhcnADzrz5UOaEucm2bBqzRUAlkhhcv3xXnpcRFWuXpk5MMN",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -29.7413463,
                    "lng": -50.0030861
                },
                "viewport": {
                    "northeast": {
                        "lat": -29.73992642010727,
                        "lng": -50.00188702010728
                    },
                    "southwest": {
                        "lat": -29.7426

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.291181,
                    "lng": 164.106817
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.28982337010727,
                        "lng": 164.1081655798927
                    },
                    "southwest": {
                        "lat": -20.29252302989272,
                        "lng": 164.1054659201072
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Malabou Beach Hotel",
            "photos": [
                {
                    "height": 2208,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/112197340787616043996\">Romain C

{
    "html_attributions": [],
    "next_page_token": "ATtYBwIBhuPNTxDkedMlRgC4MvTxXKEHltmwYOoGpDMK6QT-CkQOwE83BNG_9Hjy9HB_AXYK9mv0HQ8cTSaeK_QQvoYQXpKfBZtjC5yg2fTAvrArwwQ8l0Xz4a4kZSg3rt_DTuXPaqVgIoH-LO10ngTPnXoTYsrOFczLbRilPJLXihAbad_FkruQDD_-eMG7F4OEMbzoyUSMjJ8ROhxfJMcRGUkoP3JHxloiFJr0FqpAEHNysbEG9-0G1lQ5cMtd_qz4NFRPuLgZL3MUh0dZjTvlksMaCnjYM_H02N35zRe6AXEIChwlr7ldIOQo2zFgU9J2PyBcjLIokeZ7XSRJTkAsk_8ybzFexs7SORplf4F74MEMc7iGY45iRcTigS91Einva_HoMqcepPqmDhneQaOeaxaqy2Js8z5iBPcSpUyqqUfufmrXQdUm3ecWWtQzu2ry",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.2593709,
                    "lng": 44.307098
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.25826602010728,
                        "lng": 44.30778062989272
                    },
                    "southwest": {
                        "lat": -20.2609656

...

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJqi7Cksryyv_6JJGJ5jA240bHgT700u1jTZY5t6i7yUzlOI3-n-IgCp-DmwrPu8DGNOYsjTO05rZRq1dQMhEB15BAgmNrFzR0OSSFWdBZ6tPHAlRojlDmRF3iZu-Fz8UZ0BbEFVf83HQq_qBdG33hey2zJKQop2PH6xjMvjpBqRlqrf-9P8CrKVttQVKseqzrMak3ig2vRyK41w3X8o1lmlcFccA42iLbIxL2pNszRkNS0pBALT04HYRY9XiWZ6f0qs8RGvfyyzmf97PNaQ18Kz4buy5rwiVyzwWb_9aiPYqz66kphqd-FFrf4ELRXqqmvscnW2cQtUzvHm1XuTBqSp0_fpFsLJgYEBAyZFmBAF8Vc7_xYkEFmXq9T-sWV8YvdcaXEOccqVGz4PTLPidY7-LG7uNwEtlfFcmaL7bPsdpYba5sxH5Jcs1F6ZzLc",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 35.2067501,
                    "lng": -2.9121019
                },
                "viewport": {
                    "northeast": {
                        "lat": 35.20815157989272,
                        "lng": -2.910970770107277
                    },
                    "southwest": {
                        "lat": 35.20545192

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "CLOSED_TEMPORARILY",
            "geometry": {
                "location": {
                    "lat": 22.9096939,
                    "lng": 96.5038617
                },
                "viewport": {
                    "northeast": {
                        "lat": 22.91113002989272,
                        "lng": 96.50508842989272
                    },
                    "southwest": {
                        "lat": 22.90843037010728,
                        "lng": 96.50238877010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Mount Mogok Hotel",
            "permanently_closed": true,
            "photos": [
                {
                    "height": 607,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/map

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -15.1908754,
                    "lng": 12.1512762
                },
                "viewport": {
                    "northeast": {
                        "lat": -15.18962407010728,
                        "lng": 12.15269802989272
                    },
                    "southwest": {
                        "lat": -15.19232372989272,
                        "lng": 12.14999837010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "iu Hotel Namibe",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 4032,
                    "html_attributions": [
                        "<a 

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJF64i0cDBoVRlL8bUvAAfkHw5dw3eFng18jTA71JyHSK5qOGo4l7bu6X-485dcDb8aPwC1GXntHNtZWRPRzApDM6Mw8yuBPxPsawT6pdxQVCjcfg9IdMSlfwdkxiI1CjdX3UoSm-UAj_5_JGjDSUKhMR1E42GLq1cRAt5KMp74_lzkHXmnJEIB0ussKeHlqRP6w7KvZhxvza9AiM5HIbKN5SzA7N58jqA5MP4MXslVzcbWlMwHjHfGuZypB3yBLsYP5VrxNdrBplQtVNMLPmtgkghs3NvXbEntJuuYWamKiUAauF2px1SRME8664HrWYe9d4nkU5zdUm5yZF9a8weR9QWMSU5rcp_AVB-Z1OMrjco2Qobh2tIUKbFQp6p7uhIPiZ2lClverHrRUkMYGekB7IwyJtzOZ6w-kOydAGZtUx2sv2CFVMo67Tb97Stf",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 32.8851325,
                    "lng": 59.2167535
                },
                "viewport": {
                    "northeast": {
                        "lat": 32.88649142989272,
                        "lng": 59.21818162989271
                    },
                    "southwest": {
                        "lat": 32.883791770

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41,"Hotel Paracas, a Luxury Collection Resort, Par..."
1,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,HOTEL CHALÉ CAPITÓLIO
2,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Hotel Araçá
3,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86,Malabou Beach Hotel
4,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Cap Kimony Morondava
5,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Regenta Resort Belagavi
6,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,"Hotel Marchica Lagoon Resort, Nador Morocco"
7,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,Mount Mogok Hotel
8,391,namibe,0,AO,1558378870,74,-15.19,12.15,71.70,17.40,iu Hotel Namibe
9,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,Ghaem Hotel


In [12]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))